In [226]:
import sqlite3
import csv
from random import randint

In [227]:
con = sqlite3.connect("test.db")
cur = con.cursor()

In [228]:
def twodigitsnum(number):
    return str(number // 10) + str(number % 10)

def random_date():
    return "-".join(list(map(str, [randint(2021, 2023), twodigitsnum(randint(1, 12)), twodigitsnum(randint(1, 28))])))

def date_comparation(date1, date2):
    y1, m1, d1 = map(int, date1.split("-"))
    y2, m2, d2 = map(int, date2.split("-"))
    if y1 > y2:
        return False
    elif y1 == y2:
        if m1 > m2:
            return False
        elif m1 == m2:
            if d1 >= d2:
                return False
    return True

In [229]:
#creating books data
N = 100
with open("books.csv","w") as fout:
    dw = csv.writer(fout)
    dw.writerow(["id", "author", "title", "publish_year"])
    for i in range(1, N):
        dw.writerow([twodigitsnum(i),
                     "author" + twodigitsnum(randint(1, N // 10)),
                     "title" + twodigitsnum(i),
                     randint(1990, 2010)])

In [230]:
#creating books table from csv file
cur.execute("DROP TABLE IF EXISTS books;")
cur.execute("""
            CREATE TABLE books (
                id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                author TEXT,
                title TEXT,
                publish_year INTEGER);
            """)

with open("books.csv","r") as fin:
    dr = csv.DictReader(fin)
    to_db = [(i["author"], i["title"], int(i["publish_year"])) for i in dr]

cur.executemany("INSERT INTO books (author, title, publish_year) VALUES (?, ?, ?);", to_db)
con.commit()

In [231]:
#creating readers data
M = 30
with open("readers.csv","w") as fout:
    dw = csv.writer(fout)
    dw.writerow(["id", "name"])
    for i in range(1, M):
        dw.writerow([twodigitsnum(i), "name" + twodigitsnum(randint(1, M // 3))])

In [241]:
#creating readers table from csv file
cur.execute("DROP TABLE IF EXISTS readers;")
cur.execute("""
            CREATE TABLE readers (
                id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                name TEXT);
            """)

with open("readers.csv","r") as fin:
    dr = csv.DictReader(fin)
    to_db = [(int(i["id"]), i["name"]) for i in dr]

cur.executemany("INSERT INTO readers (id, name) VALUES (?, ?);", to_db)
con.commit()

In [234]:
#creating records data
L = 200
with open("records.csv","w") as fout:
    dw = csv.writer(fout)
    dw.writerow(["reader_id", "book_id", "taking_date", "returning_date"])
    for i in range(1, L):
        taking_date = random_date()
        returning_date = random_date()
        while not date_comparation(taking_date, returning_date):
            returning_date = random_date()
        dw.writerow([twodigitsnum(randint(1, M - 1)), twodigitsnum(randint(1, N - 1)), taking_date, returning_date])

In [235]:
#creating books table from csv file
cur.execute("DROP TABLE IF EXISTS records;")
cur.execute("""
            CREATE TABLE records (
                reader_id INTEGER,
                book_id INTEGER,
                taking_date TIMESTAMP,
                returning_date TIMESTAMP,
                FOREIGN KEY (reader_id) REFERENCES readers (id),
                FOREIGN KEY (book_id) REFERENCES books (id));
            """)

with open("records.csv","r") as fin:
    dr = csv.DictReader(fin)
    to_db = [(int(i["reader_id"]), int(i["book_id"]), i["taking_date"], i["returning_date"]) for i in dr]

cur.executemany("INSERT INTO records (reader_id, book_id, taking_date, returning_date) VALUES (?, ?, ?, ?);", to_db)
con.commit()

In [236]:
for row in cur.execute("""
                        SELECT
                            records.book_id, books.title
                        FROM records
                        INNER JOIN (
                            SELECT * FROM books
                            ) books
                        on records.book_id = books.id
                        WHERE records.returning_date > date('now')
                        ORDER BY records.book_id
                        limit 5;
                        """):
    print(row)

(1, 'title01')
(1, 'title01')
(2, 'title02')
(3, 'title03')
(4, 'title04')


In [237]:
for row in cur.execute("""
                        SELECT
                            readers.name, books.title
                        FROM records
                        INNER JOIN (
                            SELECT * FROM books
                            ) books
                        on records.book_id = books.id
                        INNER JOIN (
                            SELECT * FROM readers
                            ) readers
                        on records.reader_id = readers.id
                        ORDER BY readers.name
                        limit 5;
                        """):
    print(row)

In [238]:
for row in cur.execute("""
                        SELECT
                            author, count(id)
                        FROM books
                        GROUP BY author;
                        """):
    print(row)

('author01', 6)
('author02', 10)
('author03', 9)
('author04', 16)
('author05', 6)
('author06', 11)
('author07', 14)
('author08', 7)
('author09', 8)
('author10', 12)
